<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

'Connected: @movies.sqlite'

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql

PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [5]:
%%sql
SELECT title
FROM movie


 * sqlite:///movies.sqlite
Done.


title
12 Angry Men
The Lighthouse
Free Solo
What We Do In The Shadows
The Lord of the Rings: The Return of the King


+ Show the performance dates for one of the movies.

In [7]:
%%sql
SELECT date as "Date", start_time as "Start Time", title as "Title of Movie"
FROM screening
INNER JOIN(movie)
USING(imdb)
WHERE title = '12 Angry Men'

 * sqlite:///movies.sqlite
Done.


Date,Start Time,Title of Movie
2020-02-15,19:00,12 Angry Men
2020-02-20,14:14,Free Solo
2020-02-15,14:00,What We Do In The Shadows
2020-02-23,10:00,What We Do In The Shadows


+ Show all data concerning performances at a given theatere
  on a given date.

In [8]:
%%sql
SELECT theater_name, title, start_time, date, screening_id
FROM screening

INNER JOIN(movie)
USING(imdb)
INNER JOIN(theater)
USING(theater_name) 
WHERE theater_name = "Bergakungen" AND date = '2020-02-15'


 * sqlite:///movies.sqlite
Done.


theater_name,title,start_time,date,screening_id
Bergakungen,12 Angry Men,19:00,2020-02-15,27b6e80fce18c21b458099d127939526
Bergakungen,What We Do In The Shadows,14:00,2020-02-15,93d4aa874f6623cb34c19bb7991d9fb0


+ List all customers

In [9]:
%%sql
SELECT full_name, username
FROM customer

 * sqlite:///movies.sqlite
Done.


full_name,username
Diederik Harmsen,coolkille1996
Mats Hallström,matsa


+ List all tickets

In [17]:
%%sql
INSERT 
INTO    ticket(ticket_id, username, screening_id)
VALUES  ((lower(hex(randomblob(16)))), 'matsa', '27b6e80fce18c21b458099d127939526');

SELECT ticket_id, screening_id, title
FROM ticket
INNER JOIN(screening)
USING(screening_id)
INNER JOIN(movie)
USING(imdb)

 * sqlite:///movies.sqlite
1 rows affected.
Done.


ticket_id,screening_id,title
af6e1331b8c1468a4b58d799480014df,27b6e80fce18c21b458099d127939526,12 Angry Men
e22d053396219db152f53883129ed1d5,27b6e80fce18c21b458099d127939526,12 Angry Men
e2987d9f4a898e0030ed69a3f2a42738,27b6e80fce18c21b458099d127939526,12 Angry Men
2ddcb1202d3a1b78d2051e7a53cd1e69,27b6e80fce18c21b458099d127939526,12 Angry Men
ffce9fea7fd96d68d6f09763eba0a3b1,27b6e80fce18c21b458099d127939526,12 Angry Men


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [21]:
%%sql
INSERT 
INTO ticket(ticket_id, username, screening_id)
VALUES (lower(hex(randomblob(16))), 'matsa', '27b6e80fce18c21b458099d127939526');
PRAGMA foreign_keys = ON;

 * sqlite:///movies.sqlite
1 rows affected.
Done.


[]

In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2020), we can't do that, instead
  we can use the following idea: each row in a SQLite3 table
  has a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [ ]:
%%sql
SELECT ticket_id
FROM ticket
WHERE rowid = last_insert_rowid();

+ Try to insert two movie theaters with the same name (this
  should fail).

In [ ]:
%%sql
INSERT
INTO         theater(theater_name, capacity)
VALUES       ('Bergakungen', 23)


+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [ ]:
%%sql
INSERT 
INTO     screening(theater_name, imdb, start_time, date, screening_id)
VALUES   ('Begakungen', 'tt0050083', '12:00', '2020-02-16', '7b013fd5fb13748f125868639078d109');

+ Create a ticket where either the user or the performance
  doesn’t exist.

In [ ]:
%%sql
PRAGMA foreign_keys = OFF;
INSERT 
INTO ticket(ticket_id, username, screening_id)
VALUES (lower(hex(randomblob(16))), 'mats', '7b013fd5fb13748f125868639078d108' IS IN screening.screening_id);
PRAGMA foreign_keys = ON;